In [ ]:
import os
import csv
import requests


csv_path = "image_links.csv"
output_dir = "data"

with open(csv_path, newline="") as f:
    reader = csv.DictReader(f)
    for row in reader:
        folder = row["folder"]
        filename = row["filename"]
        github_link = row["github_link"]
        save_folder = os.path.join(output_dir, folder)
        os.makedirs(save_folder, exist_ok=True)
        save_path = os.path.join(save_folder, filename)
        if not os.path.exists(save_path):
            try:
                resp = requests.get(github_link)
                if resp.status_code == 200:
                    with open(save_path, "wb") as imgf:
                        imgf.write(resp.content)
                else:
                    print(f"Failed to download {github_link} (status {resp.status_code})")
            except Exception as e:
                print(f"Error downloading {github_link}: {e}")
        else:
            print(f"Already exists: {save_path}")

In [ ]:
LEARNING_RATE = 0.0001
BATCH_SIZE = 32
NUM_EPOCHS = 3
IMAGE_SIZE = 512

In [20]:
import tensorflow as tf

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
            directory='data',
            labels='inferred',
            label_mode='int',
            image_size=(IMAGE_SIZE, IMAGE_SIZE),
            batch_size=BATCH_SIZE,
            shuffle=True,
            seed=42,
            color_mode='rgb', #Although the images are grayscale, we need to set the color_mode to 'rgb' for the model to work properly.
            pad_to_aspect_ratio=True,
            validation_split=0.2,
            subset='training',
        )

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
            directory='data',
            labels='inferred',
            label_mode='int',
            image_size=(IMAGE_SIZE, IMAGE_SIZE),
            batch_size=BATCH_SIZE,
            shuffle=True,
            seed=42,
            color_mode='rgb',
            pad_to_aspect_ratio=True,
            validation_split=0.2,
            subset='validation',
        )                                                 

Found 1514 files belonging to 3 classes.
Using 1212 files for training.
Found 1514 files belonging to 3 classes.
Using 302 files for validation.


### Fine-Tuning ResNet

In [9]:
base_model = tf.keras.applications.ResNet50(
        weights='imagenet',
        input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
        include_top=False, #Final prediction layer will be replaced with custom one
    )

base_model.trainable = False #We will only train the custom layer we add on top of the base model

resnet = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
    tf.keras.layers.Lambda(tf.keras.applications.resnet.preprocess_input),
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3, activation='softmax', name='output')
])

In [10]:
resnet.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lambda_1 (Lambda)               │ (None, 512, 512, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 16, 16, 2048)   │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 3)              │         6,147 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,593,859 (90.00 MB)

 Trainable params: 6,147 (24.01 KB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [11]:
resnet.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
)

In [12]:
resnet.fit(train_dataset, epochs=NUM_EPOCHS)

Epoch 1/3
38/38 ━━━━━━━━━━━━━━━━━━━━ 335s 9s/step - accuracy: 0.3860 - loss: 1.1678
Epoch 2/3
38/38 ━━━━━━━━━━━━━━━━━━━━ 330s 9s/step - accuracy: 0.4986 - loss: 0.9886
Epoch 3/3
38/38 ━━━━━━━━━━━━━━━━━━━━ 330s 9s/step - accuracy: 0.6288 - loss: 0.8668


### Fine-Tuning EfficientNet

In [13]:
base_model = tf.keras.applications.EfficientNetB5(
        weights='imagenet',
        input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
        include_top=False, #Final prediction layer will be replaced with custom one
    )

base_model.trainable = False #We will only train the custom layer we add on top of the base model

efficientnet = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
    #EfficientNet has the preprocessing function built in
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3, activation='softmax', name='output')
])

115263384/115263384 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [14]:
efficientnet.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb5 (Functional)     │ (None, 16, 16, 2048)   │    28,513,527 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 3)              │         6,147 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,519,674 (108.79 MB)

 Trainable params: 6,147 (24.01 KB)

 Non-trainable params: 28,513,527 (108.77 MB)

In [15]:
efficientnet.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
)

In [16]:
efficientnet.fit(train_dataset, epochs=NUM_EPOCHS)

Epoch 1/3
38/38 ━━━━━━━━━━━━━━━━━━━━ 475s 12s/step - accuracy: 0.4263 - loss: 1.0641
Epoch 2/3
38/38 ━━━━━━━━━━━━━━━━━━━━ 455s 12s/step - accuracy: 0.5753 - loss: 0.9563
Epoch 3/3
38/38 ━━━━━━━━━━━━━━━━━━━━ 451s 12s/step - accuracy: 0.6809 - loss: 0.8672


These are just two of the many existing architectures you can use through TensorFlow. Most of them have similar syntax to create the model and then fine-tune it.

### Comparing the Two

In [19]:
print(resnet.evaluate(test_dataset, return_dict=True))
print(efficientnet.evaluate(test_dataset, return_dict=True))

10/10 ━━━━━━━━━━━━━━━━━━━━ 86s 9s/step - accuracy: 0.8039 - loss: 0.6902
{'accuracy': 0.7880794405937195, 'loss': 0.7030082941055298}
10/10 ━━━━━━━━━━━━━━━━━━━━ 113s 11s/step - accuracy: 0.8008 - loss: 0.7814
{'accuracy': 0.7913907170295715, 'loss': 0.7854565978050232}


### Save the Models

Note: Since the model files are quite large (~110 MB), they have not been pushed to Github. For model files like this, Git Large File Storage is usually used.

In [ ]:
import os

try:
    os.mkdir('models')
except FileExistsError:
    pass

resnet.save('models/finetuned_resnet.keras')
efficientnet.save('models/finetuned_efficientnet.keras')